# Testes de código

## 1. Código

In [2]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class Matriz:
    # construtor da classe, contém número de enfermeiros, número de turnos e a matriz (zerada)
    def __init__(self, nenfermeiros, nturnos, matriz = []):
        self.nenfermeiros = nenfermeiros # númeo de enfermeiros
        self.nturnos = nturnos # número de turnos
        self.enfermeiros = self.criaMatriz(nenfermeiros, nturnos) # própria matriz
        self.turnos = self.criaMatriz(nturnos, nenfermeiros) # representação diferente da matriz

# método para fazer uma matriz
    def criaMatriz(self, l, c):
        matriz = []
        for _ in range(l):
            linha = []
            for _ in range(c):
                linha.append(0)
            matriz.append(linha)
        return matriz

# retorna matriz como uma cadeia de 0s e 1s
    def printMatriz(self):
        string = ""
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                string += str(self.enfermeiros[enf][turno])
        return string

    def stringParaMatriz(self, string):
        it = 0
        for enf in range(self.nenfermeiros):
            for turno in range(self.nturnos):
                self.enfermeiros[enf][turno] = int(string[it])
                self.turnos[turno][enf] = int(string[it])
                it +=1

# função referente à restrição 1: min 1 enfermeiro e max 3 enfermeiros por turno
#  calcula o valor da penalidade e retorna
def r1(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada turno e checa se a quantidade de enfermeiros é menor que 1 ou maior que 3 para penalizar
    for turno in matriz.turnos:
        soma = sum(turno)
        if (soma < 1 or soma > 3):
            penalidade += -1

    return penalidade

# função referente à restrição 2: Cada enfermeiro deve ser alocado em 5 turnos por semana
# calcula o valor da penalidade e retorna
def r2(matriz):
    penalidade = 0 # armazena a penalidade

   # olha cada enfermeiros e checa se a quantidade de turnos que ele está é menor ou maior que 5 para penalizar
    for enfermeiro in matriz.enfermeiros:
        soma = sum(enfermeiro)
        if (soma < 5 or soma > 5):
            penalidade += -1

    return penalidade

# função referente à restrição 3: Nenhum enfermeiro pode trabalhar mais que 3 dias seguidos sem folga
# calcula o valor da penalidade e retorna
def r3(matriz):
    penalidade = 0 # armazena a penalidade
    diasTrabalhados = [] # auxiliar que verifica os dias trabalhados consecutivos
    nturno = 1 # auxiliar que verifica qual o turno

   # olha para cada turno de cada enfermeiro (em sequencia) e verifica se foi trabalhado em dias consecutivos
    for enfermeiro in matriz.enfermeiros:
        turnos = [] # auxiliar para verificar se o enfermeiro trabalhou naquele dia ou não
        for turno in enfermeiro:

           # guarda cada turno (a cada 3 turnos forma um dia)
            if (nturno <= 3):
                turnos.append(turno)

            # incrementa os dias
            if (nturno == 3) :
                nturno = 0 # garante que não passa de 3

                if (sum(turnos) > 0):
                    diasTrabalhados.append(1)
                else:
                    diasTrabalhados.append(0)
                    if (sum(diasTrabalhados) > 3):
                        penalidade += -1
                    diasTrabalhados.clear()
                turnos.clear()
            nturno += 1
        if (sum(diasTrabalhados) > 3):
                penalidade += -1

    return penalidade

# função referente à restrição 4
# calcula o valor da penalidade e retorna
def r4(matriz):
    penalidade = 0 #valor da penalidade
    turnos = 3 # turnos por dia
    enfermeiros = matriz.enfermeiros # recebe a matriz separada por enfermeiros
    for enf in range(matriz.nenfermeiros):
        for turno in range(matriz.nturnos):
            if enfermeiros[enf][turno] == 1:
                for turnoExtra in range(matriz.nturnos):
                    # verifica se dado um turno ocupado, outros turnos no mesmo horário possuem enfermeiros trabalhando
                    if (turno % turnos) != (turnoExtra % turnos) and enfermeiros[enf][turnoExtra] == 1:
                        penalidade += -1
    return penalidade

# função que calcula o fitness de determinada matriz
def fitness(matriz, restricoes):
    fit = 0
    if (1 in restricoes):
        fit += r1(matriz)
    if (2 in restricoes):
        fit += r2(matriz)
    if (3 in restricoes):
        fit += r3(matriz)
    if (4 in restricoes):
        fit += r4(matriz)
    return fit

# Cruzamento (crossover)
# contem o parâmetro var que indica qual variação está sendo usada
def crossover(tampopulacao, populacao, melhores_indices, restricoes, var=1):
    # a partir dos melhores individuos, pegamos 2 e realizamos o crossover para criar 2 novos filhos
    novos_individuos = []
    for _ in range(
        tampopulacao - len(melhores_indices)
    ):  # conta sem os melhores individuos
        index1 = random.choice(range(tampopulacao))
        index2 = random.choice(range(tampopulacao))
        while(index1 == index2):
            index2 = random.choice(range(tampopulacao))
        pai1 = populacao[index1].printMatriz()
        pai2 = populacao[index2].printMatriz()
        ponto_corte = random.randint(1, len(pai1))  # seleciona o ponto de corte
        # realiza o crossover (troca as partes de cada pai)
        filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
        filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
        if var == 1:
            matriz = Matriz(
                nenfermeiros=populacao[index1].nenfermeiros,
                nturnos=populacao[index1].nturnos,
            )
            matriz.stringParaMatriz(filho1)
            novos_individuos.append(matriz)
        elif var == 2:
            matriz = Matriz(
                nenfermeiros=populacao[index1].nenfermeiros,
                nturnos=populacao[index1].nturnos,
            )
            matriz.stringParaMatriz(filho2)
            novos_individuos.append(matriz)
    return novos_individuos

# Mutação
def mutacaoF(mutacao, novos_individuos):
   # realiza a mutação alterando um caractere em uma cadeia (se é 0, muda para 1 e vice-versa)
    for novo in novos_individuos:
        # aplica a mutação de acordo com a taxa de mutação
        if random.random() < mutacao:
            novoStr = novo.printMatriz()
            index = random.randint(0, len(novoStr) - 1)
            string = ""
            if (novoStr[index] == "0"):
                if (index == 0):
                    string = "1" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "1"
                else:
                    string = novoStr[:index] + "1" + novoStr[index+1:]
            else:
                if (index == 0):
                    string = "0" + novoStr[index+1:]
                elif index == (len(novoStr) - 1):
                    string = novoStr[:index]+ "0"
                else:
                    string = novoStr[:index] + "0" + novoStr[index+1:]
            novoStr = string
            novo.stringParaMatriz(novoStr)
    return novos_individuos

def algGenetico(nenfermeiros, nturno,restricoes, tampopulacao, interacoes, elitismo, mutacao, var):
    # cria strings aleatorias que representam um cromossomo (uma possivel configuração)
    def string_aleatoria(tamanho):
        return ''.join(random.choice('01') for _ in range(tamanho))
    # armazena a população (inicialmente vazia)
    populacao = []
    for _ in range(tampopulacao):
        matriz = Matriz(nenfermeiros=nenfermeiros, nturnos=nturno)
        matriz.stringParaMatriz(string_aleatoria(nenfermeiros*nturno)) # preenche a matriz
        populacao.append(matriz)

    for _ in range(interacoes):
        # armazena o fitness de cada elemento da população
        fitnessPopulacao = []
        fitnesspositivo = []
        for matriz in populacao:
            value = fitness(matriz,restricoes)
            fitnessPopulacao.append(value)
            fitnesspositivo.append(value *(-1))
        novos_individuos= populacao
        melhores_individuos = []
        if (elitismo > 0):
          # escolha dos melhores indivíduos (elitismo)
          melhores_indices = np.argsort(fitnesspositivo)[:int(elitismo * tampopulacao)] # pega os indices dos melhores fitness

          for indice in melhores_indices:
              melhores_individuos.append(populacao[indice]) # pega os elementos correspondentes aos indices

          # realiza o crossover
          novos_individuos = crossover(tampopulacao=tampopulacao, melhores_indices=melhores_indices, restricoes=restricoes, var=1, populacao=populacao)

        # realiza a mutação
        novos_individuos = mutacaoF(mutacao=mutacao, novos_individuos=novos_individuos)
        for individuo in novos_individuos:
            melhores_individuos.append(individuo)
        populacao = melhores_individuos

    # Encontrando a melhor solução
    melhor_indice = np.argmax(fitnessPopulacao)
    melhor_solucao = populacao[melhor_indice]
    return melhor_solucao.printMatriz(), fitness(melhor_solucao, restricoes)

## 2. Bloco de Experimentação 1 (Valores de Elitismo: 0, 0.1, 0.25, 0.5, 0.75)

### 2.1. Variação do crossover: 1

In [ ]:
#leitura dos dados
nenfermeiros = 10

nturnos = 21

restricoes = [1,2,3,4]

tampopulacao = 100

interacoes = 1000

mutacao = 0.1

var = 1

# Valores de Elitismo para o teste
elitismos = [0,0.1,0.25,0.5,0.75]

# DataFrame para visualização dos dados
lista = []

# Armazenando as listas que vão ser o dataframe
for _ in range(10):
  lista.append([])

# Execução e armazenamento dos dados
colunas = []
for nel in range(len(elitismos)):
  colunas.append(f"Elitismo {elitismos[nel]}")
  for linha in range(10):
    _, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tampopulacao, interacoes=interacoes, elitismo=elitismos[nel], mutacao=mutacao, var= var)
    lista[linha].append(fit)

df1 = pd.DataFrame(lista, columns=colunas)
display(df1)

,Elitismo 0,Elitismo 0.1,Elitismo 0.25,Elitismo 0.5,Elitismo 0.75
0,-484,-4,-4,-7,-8
1,-509,-6,-6,-8,-7
2,-570,-5,-4,-5,-5
3,-517,-3,-5,-5,-7
4,-578,-5,-6,-5,-9
5,-514,-7,-6,-5,-8
6,-500,-7,-5,-6,-10
7,-527,-4,-4,-9,-8
8,-553,-5,-2,-8,-8
9,-514,-6,-6,-3,-6


- Dicionário de Dados

In [ ]:
df1_dict = { 'Elitismo 0': [-484, -509, -570,-517,-578,-514, -500,-527, -553,-514],
      'Elitismo 0.1':[-4,-6,-5,-3,-5,-7,-7,-4,-5,-6],
       'Elitismo 0.25':[-4,-6,-4,-5,-6,-6,-5,-4,-2,-6],
       'Elitismo 0.5':[-7,-8,-5,-5,-5,-5,-6,-9,-8,-3],
       'Elitismo 0.75':[-8,-7,-5,-7, -9,-8,-10,-8,-8,-6]}
df = pd.DataFrame(df1_dict)

- Descrição geral dos dados

In [ ]:
display(df.describe())

,Elitismo 0,Elitismo 0.1,Elitismo 0.25,Elitismo 0.5,Elitismo 0.75
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,-526.600000,-5.200000,-4.800000,-6.100000,-7.600000
std,30.667391,1.316561,1.316561,1.852926,1.429841
min,-578.000000,-7.000000,-6.000000,-9.000000,-10.000000
25%,-546.500000,-6.000000,-6.000000,-7.750000,-8.000000
50%,-515.500000,-5.000000,-5.000000,-5.500000,-8.000000
75%,-510.250000,-4.250000,-4.000000,-5.000000,-7.000000
max,-484.000000,-3.000000,-2.000000,-3.000000,-5.000000


Segundo a análise, o melhor elitismo foi o de valor 0.25. O valor menor foi -6 e o maior foi -2. A média foi a maior, -4.8. E o desvio padrão foi o de menor valor, visto que o elitismo de 0.1 também possui o mesmo valor de desvio padrão.

### 2.2. Variação do crossover: 2

In [ ]:
#leitura dos dados
nenfermeiros = 10

nturnos = 21

restricoes = [1,2,3,4]

tampopulacao = 100

interacoes = 1000

mutacao = 0.1

var = 2

# Valores de Elitismo para o teste
elitismos = [0,0.1,0.25,0.5,0.75]

# DataFrame para visualização dos dados
lista = []

# Armazenando as listas que vão ser o dataframe
for _ in range(10):
  lista.append([])

# Execução e armazenamento dos dados
colunas = []
for nel in range(len(elitismos)):
  colunas.append(f"Elitismo {elitismos[nel]}")
  for linha in range(10):
    _, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tampopulacao, interacoes=interacoes, elitismo=elitismos[nel], mutacao=mutacao, var= var)
    lista[linha].append(fit)

df2 = pd.DataFrame(lista, columns=colunas)
display(df2)

,Elitismo 0,Elitismo 0.1,Elitismo 0.25,Elitismo 0.5,Elitismo 0.75
0,-536,-6,-6,-6,-9
1,-524,-4,-5,-5,-8
2,-388,-4,-8,-6,-6
3,-559,-6,-5,-6,-8
4,-441,-4,-7,-6,-8
5,-540,-6,-6,-5,-7
6,-518,-5,-5,-5,-7
7,-517,-5,-5,-6,-8
8,-484,-5,-7,-5,-7
9,-557,-7,-5,-6,-6


- Dicionário de Dados

In [4]:
df2_dict = { 'Elitismo 0': [-536, -524, -388,-559,-441,-540, -518,-517, -484,-557],
      'Elitismo 0.1':[-6,-4,-4,-6,-4,-6,-5,-5,-5,-7],
       'Elitismo 0.25':[-6,-5,-8,-5,-7,-6,-5,-5,-7,-5],
       'Elitismo 0.5':[-6,-5,-6,-6,-6,-5,-5,-6,-5,-6],
       'Elitismo 0.75':[-9,-8,-6,-8, -8,-7,-7,-8,-7,-6]}
df = pd.DataFrame(df2_dict)

- Descrição Geral dos Dados

In [5]:
display(df.describe())

,Elitismo 0,Elitismo 0.1,Elitismo 0.25,Elitismo 0.5,Elitismo 0.75
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,-506.400000,-5.200000,-5.900000,-5.600000,-7.400000
std,54.453242,1.032796,1.100505,0.516398,0.966092
min,-559.000000,-7.000000,-8.000000,-6.000000,-9.000000
25%,-539.000000,-6.000000,-6.750000,-6.000000,-8.000000
50%,-521.000000,-5.000000,-5.500000,-6.000000,-7.500000
75%,-492.250000,-4.250000,-5.000000,-5.000000,-7.000000
max,-388.000000,-4.000000,-5.000000,-5.000000,-6.000000


Para essa segunda variação do crossover, o elitismo melhor foi o de valor 0.1, visto que sua média foi a maior, seus valores máx e min foram, respectivamente, -4 e -7. Porém, seu desvio padrão foi o o terceiro menor. O elitismo de valor 0.5 também obteve bons resultados, já que seus valore de máx e min foram, respectivamente, -5 e -6, seu desvio padrão foi o mais baixo e ainda sua média foi de -5.6, a segunda média mais alta.

## 3. Bloco de Experimentação 2 (Tamanhos da população: 10, 25, 50, 100, 500, 1000)

### 3.1. Variação de crossover: 1

In [ ]:
#leitura dos dados
nenfermeiros = 10

nturnos = 21

restricoes = [1,2,3,4]

# Valores de tamanhos da população para o teste
tamanhos = [10, 25, 50, 100, 500, 1000]

interacoes = 1000

mutacao = 0.1

var = 1

elitismo = 0.1

# DataFrame para visualização dos dados
lista = []

# Armazenando as listas que vão ser o dataframe
for _ in range(10):
  lista.append([])

# Execução e armazenamento dos dados
colunas = []
for tam in range(len(tamanhos)):
  colunas.append(f"Tamanho {tamanhos[tam]}")
  for linha in range(10):
    _, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tamanhos[tam], interacoes=interacoes, elitismo=elitismo, mutacao=mutacao, var= var)
    lista[linha].append(fit)

df1 = pd.DataFrame(lista, columns=colunas)
display(df1)

,Tamanho 10,Tamanho 25,Tamanho 50,Tamanho 100,Tamanho 500,Tamanho 1000
0,-18,-8,-8,-4,-2,-1
1,-17,-9,-5,-2,-1,-1
2,-14,-7,-6,-4,-2,-2
3,-20,-9,-7,-5,-2,-1
4,-22,-7,-4,-7,-4,-1
5,-30,-7,-10,-5,-2,-2
6,-11,-9,-6,-3,-1,-3
7,-22,-9,-6,-5,-3,-2
8,-20,-8,-5,-7,-2,-2
9,-16,-17,-8,-7,-4,-3


- Dicionário de Dados

In [6]:
df1_dict = { 'Tamanho 10': [-18, -17, -14,-20,-22,-30, -11,-22, -20,-16],
      'Tamanho 25':[-8,-9,-7,-9,-7,-7,-9,-9,-8,-17],
       'Tamanho 50':[-8,-5,-6,-7,-4,-10,-6,-6,-5,-8],
       'Tamanho 100':[-4,-2,-4,-5,-7,-5,-3,-5,-7,-7],
       'Tamanho 500':[-2,-1,-2,-2, -4,-2,-1,-3,-2,-4],
        'Tamanho 1000':[-1,-1,-2,-1, -1,-2,-3,-2,-2,-3]}
df = pd.DataFrame(df1_dict)

- Descrição Geral dos Dados

In [7]:
display(df.describe())

,Tamanho 10,Tamanho 25,Tamanho 50,Tamanho 100,Tamanho 500,Tamanho 1000
count,10.000000,10.00000,10.000000,10.00000,10.00000,10.000000
mean,-19.000000,-9.00000,-6.500000,-4.90000,-2.30000,-1.800000
std,5.206833,2.94392,1.779513,1.72884,1.05935,0.788811
min,-30.000000,-17.00000,-10.000000,-7.00000,-4.00000,-3.000000
25%,-21.500000,-9.00000,-7.750000,-6.50000,-2.75000,-2.000000
50%,-19.000000,-8.50000,-6.000000,-5.00000,-2.00000,-2.000000
75%,-16.250000,-7.25000,-5.250000,-4.00000,-2.00000,-1.000000
max,-11.000000,-7.00000,-4.000000,-2.00000,-1.00000,-1.000000


### 3.2. Variação de crossover: 2

In [ ]:
#leitura dos dados
nenfermeiros = 10

nturnos = 21

restricoes = [1,2,3,4]

# Valores de tamanhos da população para o teste
tamanhos = [10, 25, 50, 100, 500, 1000]

interacoes = 1000

mutacao = 0.1

var = 2

elitismo = 0.1

# DataFrame para visualização dos dados
lista = []

# Armazenando as listas que vão ser o dataframe
for _ in range(10):
  lista.append([])

# Execução e armazenamento dos dados
colunas = []
for tam in range(len(tamanhos)):
  colunas.append(f"Tamanho {tamanhos[tam]}")
  for linha in range(10):
    _, fit = algGenetico(nenfermeiros=nenfermeiros, nturno=nturnos, restricoes=restricoes, tampopulacao=tamanhos[tam], interacoes=interacoes, elitismo=elitismo, mutacao=mutacao, var= var)
    lista[linha].append(fit)

df2 = pd.DataFrame(lista, columns=colunas)
display(df2)

,Tamanho 10,Tamanho 25,Tamanho 50,Tamanho 100,Tamanho 500,Tamanho 1000
0,-12,-15,-6,-4,-3,-2
1,-23,-7,-6,-3,-2,-2
2,-17,-8,-7,-6,-3,-2
3,-18,-8,-6,-5,-3,-1
4,-16,-5,-6,-3,-4,-2
5,-14,-7,-7,-5,-1,-2
6,-9,-8,-8,-4,-2,-2
7,-14,-7,-7,-5,-2,-2
8,-31,-9,-7,-4,-1,-1
9,-21,-8,-6,-4,-2,-2


- Dicionário de Dados

In [8]:
df2_dict = { 'Tamanho 10': [-12, -23, -17,-18,-16,-14, -9,-14, -31,-21],
      'Tamanho 25':[-15,-7,-8,-8,-5,-7,-8,-7,-9,-8],
       'Tamanho 50':[-6,-6,-7,-6,-6,-7,-8,-7,-7,-6],
       'Tamanho 100':[-4,-3,-6,-5,-3,-5,-4,-5,-4,-4],
       'Tamanho 500':[-3,-2,-3,-3, -4,-1,-2,-2,-1,-2],
        'Tamanho 1000':[-2,-2,-2,-1, -2,-2,-2,-2,-1,-2]}
df = pd.DataFrame(df2_dict)

- Descrição Geral dos Dados

In [9]:
display(df.describe())

,Tamanho 10,Tamanho 25,Tamanho 50,Tamanho 100,Tamanho 500,Tamanho 1000
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,-17.500000,-8.200000,-6.600000,-4.300000,-2.300000,-1.800000
std,6.276057,2.616189,0.699206,0.948683,0.948683,0.421637
min,-31.000000,-15.000000,-8.000000,-6.000000,-4.000000,-2.000000
25%,-20.250000,-8.000000,-7.000000,-5.000000,-3.000000,-2.000000
50%,-16.500000,-8.000000,-6.500000,-4.000000,-2.000000,-2.000000
75%,-14.000000,-7.000000,-6.000000,-4.000000,-2.000000,-2.000000
max,-9.000000,-5.000000,-6.000000,-3.000000,-1.000000,-1.000000
